## Prerequisites
### Couchbase Configuration
Follow the steps in [01-setup.ipynb](01-setup.ipynb) to configure the couchbase bucket, load the data into it, and create the spatial view.
### Enable the ArcGIS jupyter notebook widgets
Run these commands before starting the jupyter notebook server for the first time - the notebook server should be **stopped**.
```
$ jupyter nbextension install --py --sys-prefix arcgis
$ jupyter nbextension enable --py --sys-prefix arcgis
```

In [1]:
from arcgis.gis import GIS

/home/josh/.local/share/virtualenvs/cb-geo-demo-3Q7hPktG/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/josh/.local/share/virtualenvs/cb-geo-demo-3Q7hPktG/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
gis = GIS()

In [3]:
map = gis.map("San Antonio, TX 78229")
map.basemap = 'osm'
map.zoom = 14

In [4]:
map

MapView(basemap='osm', layout=Layout(height='400px', width='100%'), zoom=14.0)

In [5]:
map.extent

{'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
 'xmin': -10981839.090819877,
 'ymin': 3437306.0202543377,
 'xmax': -10963513.313288528,
 'ymax': 3444949.7230828446}

In [6]:
xmin = map.extent['xmin']
ymin = map.extent['ymin']
xmax = map.extent['xmax']
ymax = map.extent['ymax']

In [7]:
import pyproj

In [8]:
# For some reason this is the only way to initialize Web Mercator
srcProj = pyproj.Proj(init='epsg:3857')
# Can also be initialized like so:
# Proj(proj="longlat", ellps="WGS84", datum="WGS84")
dstProj = pyproj.Proj(init='epsg:4326')

In [9]:
lower_left = pyproj.transform(srcProj, dstProj, xmin, ymin)
upper_right = pyproj.transform(srcProj, dstProj, xmax, ymax)

In [10]:
lower_left

(-98.65153903024724, 29.483313548576277)

In [11]:
upper_right

(-98.4869157697494, 29.54306834103877)

In [12]:
from cbgeo.utils import connect
from cbgeo.query import query_bounding_box

In [13]:
bucket = connect()

In [14]:
view = query_bounding_box(bucket, lower_left, upper_right)

In [15]:
from arcgis.geometry import Point

In [16]:
map

MapView(basemap='osm', jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True, zoo…

In [17]:
# Iterate over the points in the couchbase result set and draw them
# on the map with popups.
for row in view:
    coordinates = row.geometry['coordinates']
    x_coord = coordinates[0]
    y_coord = coordinates[1]
    # No need to re-project the coordinates, since the map widget
    # can project on the fly.
    point = Point(x=x_coord, y=y_coord, spatialReference=4326)
    attrs = row.value
    # Create the popup title using the LOCATION field, truncating the
    # text so it doesn't over-extend the title bar.
    title = attrs['LOCATION'][:26]
    # Create the HTML popup content, using the feature attributes from couchbase.
    content = "<br>".join(["<strong>{0}</strong>: {1}".format(k,v) for k,v in attrs.items()])
    popup_dict = {'title': title, 'content': content}
    map.draw(point, popup=popup_dict)